In [3]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl==0.15.2 triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
    !pip install --no-deps unsloth

In [5]:
if True:
    from unsloth import FastLanguageModel
    tuned_model, tuned_tokenizer = FastLanguageModel.from_pretrained(
        model_name = "jackf857/canton_fine-tuned_qwen3b-instruct",
        max_seq_length = 2048,
        dtype = None,
        load_in_4bit = True,
    )
    FastLanguageModel.for_inference(tuned_model) # Enable native 2x faster inference

alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""


==((====))==  Unsloth 2025.3.19: Fast Qwen2 patching. Transformers: 4.51.3.
   \\   /|    NVIDIA L4. Num GPUs = 1. Max memory: 22.161 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.9. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.36G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/271 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/7.36k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

...

added_tokens.json:   0%|          | 0.00/605 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/120M [00:00<?, ?B/s]

Unsloth 2025.3.19 patched 36 layers with 36 QKV layers, 36 O layers and 36 MLP layers.


In [7]:
inputs = tuned_tokenizer(
[
    alpaca_prompt.format(
        "我想寫一封投訴信畀某間公司，話佢哋嘅客服服務好差，話佢哋點樣對待我，仲有最後我想要咩補償.", # instruction
       """我早前在貴公司買咗一部電腦，但係佢係壞嘅。我試過聯絡你哋嘅客服，但係佢哋嘅服務態度好差，冇人肯幫我解決問題。
        我想要一個退款或者換一部新嘅電腦。我喺 2022 年 8 月 28 日喺貴公司嘅官網買咗一部電腦。張單號係 1234567890。
        我喺 2022 年 8 月 31 日收到部電腦，但係發現佢係壞嘅。我即刻聯絡你哋嘅客服，佢哋叫我寄返部電腦畀佢哋。
        我喺 2022 年 9 月 5 日寄返部電腦畀佢哋。一直到 2022 年 9 月 15 日，我都仲未收到佢哋嘅答覆。
        我再打畀佢哋嘅客服追問，佢哋話佢哋仲未收到我寄返嘅電腦。我話畀佢哋知道我有張郵寄收據，可以證明我已經寄返部電腦畀佢哋。
        但係佢哋話佢哋唔接受郵寄收據，佢哋只接受佢哋自己嘅快遞單據。我話畀佢哋知道佢哋咁樣係唔合理嘅，
        但係佢哋堅持要我哋自己俾錢寄返部電腦畀佢哋。我感到非常無奈同埋憤怒。我希望貴公司可以畀一個退款或者換一部新嘅電腦畀我，並且作出合適嘅賠償。""",# input
        "", # output - leave this blank for generation!    # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tuned_tokenizer)
_ = tuned_model.generate(**inputs, streamer = text_streamer, max_new_tokens = 1024, use_cache = False)

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
我想寫一封投訴信畀某間公司，話佢哋嘅客服服務好差，話佢哋點樣對待我，仲有最後我想要咩補償.

### Input:
我早前在貴公司買咗一部電腦，但係佢係壞嘅。我試過聯絡你哋嘅客服，但係佢哋嘅服務態度好差，冇人肯幫我解決問題。
        我想要一個退款或者換一部新嘅電腦。我喺 2022 年 8 月 28 日喺貴公司嘅官網買咗一部電腦。張單號係 1234567890。
        我喺 2022 年 8 月 31 日收到部電腦，但係發現佢係壞嘅。我即刻聯絡你哋嘅客服，佢哋叫我寄返部電腦畀佢哋。
        我喺 2022 年 9 月 5 日寄返部電腦畀佢哋。一直到 2022 年 9 月 15 日，我都仲未收到佢哋嘅答覆。
        我再打畀佢哋嘅客服追問，佢哋話佢哋仲未收到我寄返嘅電腦。我話畀佢哋知道我有張郵寄收據，可以證明我已經寄返部電腦畀佢哋。
        但係佢哋話佢哋唔接受郵寄收據，佢哋只接受佢哋自己嘅快遞單據。我話畀佢哋知道佢哋咁樣係唔合理嘅，
        但係佢哋堅持要我哋自己俾錢寄返部電腦畀佢哋。我感到非常無奈同埋憤怒。我希望貴公司可以畀一個退款或者換一部新嘅電腦畀我，並且作出合適嘅賠償。

### Response:
尊敬嘅貴公司，

我喺 2022 年 8 月 28 日喺貴公司嘅官網購買咗一部電腦，但係佢係壞嘅。我試過聯絡你哋嘅客服，但係佢哋嘅服務態度好差，冇人肯幫我解決問題。

我喺 2022 年 8 月 31 日收到部電腦，但係發現佢係壞嘅。我即刻聯絡你哋嘅客服，佢哋叫我寄返部電腦畀佢哋。

我喺 2022 年 9 月 5 日寄返部電腦畀佢哋。一直到 2022 年 9 月 15 日，我都仲未收到佢哋嘅答覆。

我再打畀佢哋嘅客服追問，佢哋話佢哋仲未收到我寄返嘅電腦。我話畀佢哋知道我有張郵寄收據，可以證明我已經寄返部電腦畀佢哋。

但係佢哋話佢哋唔接受郵寄收據，佢哋只接受佢哋自

In [24]:
inputs = tuned_tokenizer(
[
    alpaca_prompt.format(
        "有人想買樓，佢想你畀啲意見佢，佢應該係香港邊區買樓會好啲？", # instruction
        "", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tuned_tokenizer)
_ = tuned_model.generate(**inputs, streamer = text_streamer, max_new_tokens = 1024, use_cache=False)

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
有人想買樓，佢想你畀啲意見佢，佢應該係香港邊區買樓會好啲？

### Input:


### Response:
根據我嘅經驗，香港邊區買樓，有好多因素要考慮。例如，樓價、租金、交通、生活成本等。如果你喺香港嘅中環或者尖沙咀買樓，可能租金就比較貴，但交通方便，生活便利。如果你喺香港嘅邊區買樓，可能租金比較便宜，但交通不便，生活不方便。所以，你需要根據自己嘅需要和預算，選擇最合適嘅地區買樓。<|im_end|>


In [25]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "Qwen/Qwen2.5-3B-Instruct",
    max_seq_length = 2048,
    dtype = None,
    load_in_4bit = True,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

==((====))==  Unsloth 2025.3.19: Fast Qwen2 patching. Transformers: 4.51.3.
   \\   /|    NVIDIA L4. Num GPUs = 1. Max memory: 22.161 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.9. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [26]:
import pandas as pd
from transformers import TextStreamer
from datasets import load_dataset

# Load your original and fine-tuned models and tokenizers
# Assume you have already loaded them as 'model', 'tokenizer', 'tuned_model', and 'tuned_tokenizer'

# Get the last 10 samples from the training dataset
dataset = load_dataset("hon9kon9ize/yue-alpaca", split="train")
eval_dataset = dataset[-10:]

alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

eval_dataset


{'instruction': ['我想整一個新嘅花藝擺設，你會點樣幫我揀花材？',
  '我想做一個關於環保嘅海報，你會點樣幫我揀主題？',
  '我想寫一個關於友誼嘅故事，你會點樣幫我寫結尾？',
  '如果要形容你人生嘅愛情觀，你會用邊個詞語？點解？',
  '將呢段文字翻譯成中文。',
  '喺唔更改句子結構嘅情況下，喺句子入面加入「雖然」一詞。',
  '將以下呢個句子嘅語氣改為高興。',
  '根據以下資訊，寫一篇關於呢個產品嘅評論。',
  '改寫以下呢段對話，採用嚴肅嘅語氣。',
  '將呢段對話改寫成正式嘅語氣。'],
 'input': ['<noinput>',
  '<noinput>',
  '<noinput>',
  '<noinput>',
  'I am a large language model, trained by Google.',
  '佢好鍾意食蛋糕。',
  '「我唔想見到你。」',
  '呢隻產品好靚，質量又好，仲好平，真係抵買！',
  'A：你依家做緊乜嘢啊？\nB：我喺睇漫畫呀！',
  'A：你依家做緊乜嘢啊？\nB：我喺煲劇呀！'],
 'output': ['可以選擇一些顏色鮮豔嘅花材，例如係玫瑰、百合或者鬱金香。另外，亦可以選擇一些花期長嘅花材，例如係康乃馨或者非洲菊。',
  '可以選擇一些與環保相關嘅主題，例如係「節約能源」、「減少碳排放」或者「保護海洋」。另外，亦可以選擇一些與環保相關嘅口號，例如係「綠色生活，低碳出行」或者「愛護地球，人人有責」。',
  '可以寫下男女主角經歷過風風雨雨，最終成為咗一生一世嘅好朋友。另外，亦可以寫下男女主角各自成長，但係佢哋嘅友誼依然不變。',
  '「忠誠」。因為愛情唔係佔有，而係陪伴。',
  '我係一個由 Google 訓練嘅大型語言模型。',
  '佢雖然咁鍾意食蛋糕，但係佢好少食。',
  '「好耐冇見到你啦，我好開心呀！」',
  '呢隻產品無論喺外觀設計、製作質素，定係價格方面，都無懈可擊。絕對係物超所值，值得推薦！',
  'A：你此刻正在進行何種活動？\nB：我正觀看漫畫。',
  'A：請告知閣下當前正在從事何種活動。\nB：我正觀看電視劇。']}

In [27]:
# Function to generate responses from both models
def generate_responses(examples):
    results = []
    for i in range(len(examples['instruction'])): # Iterate by index
        instruction = examples["instruction"][i]
        input_text = examples["input"][i]

        # Generate response from the original model
        inputs = tokenizer(
            [alpaca_prompt.format(instruction, input_text, "")],
            return_tensors="pt",
        ).to("cuda")
        text_streamer = TextStreamer(tokenizer)
        original_response = model.generate(
            **inputs, streamer=text_streamer, max_new_tokens=256, use_cache=False
        )
        original_response = tokenizer.decode(original_response[0])

        # Generate response from the fine-tuned model
        inputs = tuned_tokenizer(
            [alpaca_prompt.format(instruction, input_text, "")],
            return_tensors="pt",
        ).to("cuda")
        text_streamer = TextStreamer(tuned_tokenizer)
        tuned_response = tuned_model.generate(
            **inputs, streamer=text_streamer, max_new_tokens=256, use_cache=False
        )
        tuned_response = tuned_tokenizer.decode(tuned_response[0])

        results.append(
            {
                "instruction": instruction,
                "input": input_text,
                "original_response": original_response,
                "tuned_response": tuned_response,
            }
        )
    return results

# Generate responses for the evaluation dataset
eval_results = generate_responses(eval_dataset)

# Create a Pandas DataFrame for human evaluation
eval_df = pd.DataFrame(eval_results)

# Save the DataFrame to a CSV file
eval_df.to_csv("human_evaluation.csv", index=False)

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
我想整一個新嘅花藝擺設，你會點樣幫我揀花材？

### Input:
<noinput>

### Response:
為您設計一組新的花藝擺設時，選擇花材是非常重要的一步。以下是一些挑選花材的建議：

1. **考慮主題和氛圍**：首先確定您的花藝擺設想要傳達的主題或要營造的氛圍。是浪漫、自然、節日還是其他風格？不同的主題需要不同風格的花材。

2. **考慮季節因素**：某些花材在特定季節表現最佳，因此了解季節性花材也很重要。例如，春天可以使用玫瑰、郁金香和洋蘭，秋天則可考慮向日葵、菊花和紅掌。

3. **考慮花材的顏色和搭配**：不同顏色的花材可以創造不同的效果。例如，紅色和粉紅色的花材可以營造出浪漫和溫暖的感覺；而白色和藍色的花材則更適合創造純淨和平靜的感覺。

4. **考慮花材的生長特性**：不同的花材有不同的生長特性和照顧方式。有些花材需要充足的陽光，有些則需要低光環境。請確保所選花材的生長需求與擺設地點相匹配。


Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
我想整一個新嘅花藝擺設，你會點樣幫我揀花材？

### Input:
<noinput>

### Response:
要整一個新嘅花藝擺設，可以考慮以下幾種花材：
- 紅玫瑰：紅玫瑰代表愛情，佢哋喺花藝擺設中經常出現。紅玫瑰可以同其他花材組合咁用，例如同康乃馨、百合、牡丹等等。
- 百合：百合喺花藝擺設中經常出現，佢哋代表純潔與高貴。百合可以同其他花材組合咁用，例如同康乃馨、玫瑰、紫羅蘭等等。
- 玫瑰：玫瑰喺花藝擺設中經常出現，佢哋代表愛情、

In [32]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, HTML

# Load the DataFrame
eval_df = pd.read_csv("human_evaluation.csv")

# Create a new column for user scores, initializing with NaN
eval_df['user_score'] = float('nan')

# Function to display content and get user score for a single row
def get_user_score(row_index):
    instruction = eval_df.loc[row_index, 'instruction']
    input_text = eval_df.loc[row_index, 'input']
    original_response = eval_df.loc[row_index, 'original_response']
    tuned_response = eval_df.loc[row_index, 'tuned_response']

    display(HTML(f"<h3>Instruction:</h3><p>{instruction}</p>"))
    display(HTML(f"<h3>Input:</h3><p>{input_text}</p>"))
    display(HTML(f"<h3>Original Response:</h3><p>{original_response}</p>"))
    display(HTML(f"<h3>Tuned Response:</h3><p>{tuned_response}</p>"))

    score_widget = widgets.IntSlider(min=1, max=10, step=1, description="Score (1-10):")
    display(score_widget)

    def update_score(change):
        eval_df.loc[row_index, 'user_score'] = change.new
        print(f"Score recorded: {change.new}")
        # Close the widget after input

    score_widget.observe(update_score, names='value')


# Iterate through each row and get user score
for i in range(len(eval_df)):
    get_user_score(i)

# Calculate and display average score after all inputs
def calculate_and_display_average():
    average_score = eval_df['user_score'].mean()
    print(f"The average user score is: {average_score}")



IntSlider(value=1, description='Score (1-10):', max=10, min=1)

IntSlider(value=1, description='Score (1-10):', max=10, min=1)

IntSlider(value=1, description='Score (1-10):', max=10, min=1)

IntSlider(value=1, description='Score (1-10):', max=10, min=1)

IntSlider(value=1, description='Score (1-10):', max=10, min=1)

...

IntSlider(value=1, description='Score (1-10):', max=10, min=1)

IntSlider(value=1, description='Score (1-10):', max=10, min=1)

IntSlider(value=1, description='Score (1-10):', max=10, min=1)

IntSlider(value=1, description='Score (1-10):', max=10, min=1)

Score recorded: 2
Score recorded: 7
Score recorded: 8
Score recorded: 7
Score recorded: 2
Score recorded: 1
Score recorded: 2
Score recorded: 3
Score recorded: 7
Score recorded: 9
Score recorded: 8
Score recorded: 9
Score recorded: 2
Score recorded: 3
Score recorded: 4
Score recorded: 2
Score recorded: 10
Score recorded: 2
Score recorded: 10
Score recorded: 2
Score recorded: 10
Score recorded: 2
Score recorded: 6
Score recorded: 7
Score recorded: 6
Score recorded: 2


In [33]:
calculate_and_display_average()

The average user score is: 6.555555555555555
